In [1]:
import pandas as pd
import os
import glob
import numpy as np
from sqlalchemy import create_engine

In [8]:
def extract_data(data_dir:str) -> pd.DataFrame:
    # extract and combine all csv files.
    all_files = glob.glob(os.path.join(data_dir , "*.csv"))
    li = []
    for filename in all_files:
        df = pd.read_csv(filename)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [9]:
data_directory = r'.\data\products'
# E

extracted_data = extract_data(data_directory)

In [10]:
extracted_data

,link,id,title,images,member_since,description,owner,reviews,google_maps_locatoin_link,long,...,Bathrooms,Furnished?,Surface Area,Floor,Building Age,Main Amenities,Reference ID,Number of Floors,Additional Amenities,Property Status
0,https://opensooq.com/en/search/244260887/resid...,244260887.0,أرض للبيع ناعور - الروضه,['https://opensooq-images.os-cdn.com/previews/...,17-05-2015,دونم ارض إستثماري في منطقة الروضة ناعور حوض 5 ...,مكتب العال و الروضه العقاري,( 12 ),https://www.google.com/maps/search/?api=1&quer...,31.831527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://opensooq.com/en/search/243608259/194-m...,243608259.0,شقة طابق اخير مع روف مميزة,['https://opensooq-images.os-cdn.com/previews/...,12-08-2018,شقة طابق الاخير مع روف الشقه 161 مترالروف 33 م...,الكسواني للاسكان,( 18 ),https://www.google.com/maps/search/?api=1&quer...,32.024303,...,3,Unfurnished,194 meter square,Third Floor,0 - 11 months,"Balcony, Laundry Room, Maid Room, Double Glaze...",NaN,NaN,NaN,NaN
2,https://opensooq.com/en/search/244651225/77-m2...,244651225.0,مشروع جبل عمان فندق حياه عمان شقة سياحية من ...,['https://opensooq-images.os-cdn.com/previews/...,14-03-2016,مشروع جبل عمان فندق حياه عمان شقة سياحية من ...,شركه رائد العساف وشريكه,( 8 ),https://www.google.com/maps/search/?api=1&quer...,31.892895,...,3,Unfurnished,77 meter square,Third Floor,0 - 11 months,"Air Conditioning, Heating, Balcony, Maid Room,...",0799118880,NaN,NaN,NaN
3,https://opensooq.com/en/search/244509387/resid...,244509387.0,قطعة ارض سكنية مميزة جدا ومطلة للبيع على شارع...,['https://opensooq-images.os-cdn.com/previews/...,18-09-2021,قطعة ارض مميزة جدا بأرقى احياء ابو نصير فلل م...,بلال نجم,( 0 ),https://www.google.com/maps/search/?api=1&quer...,32.066395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://opensooq.com/en/search/244715537/191-m...,244715537.0,شقة طابق اخير مع روف,['https://opensooq-images.os-cdn.com/previews/...,12-08-2018,شقة طابق اخير مع روف مميزة جدا تملك شقة أحلامك...,الكسواني للاسكان,( 18 ),https://www.google.com/maps/search/?api=1&quer...,32.044338,...,3,Unfurnished,191 meter square,Last floor with roof,0 - 11 months,"Electric Shutters, Balcony, Laundry Room, Doub...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,https://opensooq.com/en/search/244514339/comme...,244514339.0,الشميساني - تنظيم مكاتب -مقابل العبدلي بوليفار...,['https://opensooq-images.os-cdn.com/previews/...,09-11-2014,محافظه العاصمه - عمان- .الشميساني - حوض 14 الل...,F J T,( 6 ),https://www.google.com/maps/search/?api=1&quer...,31.969069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1496,https://opensooq.com/en/search/245098823/175-m...,245098823.0,22451 شقة مميزة كردور عبدون حجار النوابلسه ش ...,['https://opensooq-images.os-cdn.com/previews/...,31-10-2015,22451 شقة مميزة كردور عبدون حجار النوابلسه ش ...,جيفارا العقاري,( 820 ),https://www.google.com/maps/search/?api=1&quer...,31.970973,...,3,Unfurnished,175 meter square,Semi Ground Floor,1 - 5 years,"Air Conditioning, Heating, Balcony, Maid Room,...",22451,NaN,NaN,NaN
1497,https://opensooq.com/en/search/241357913/30-m2...,241357913.0,استديو للبيع مباشر من المالك,['https://opensooq-images.os-cdn.com/previews/...,03-03-2016,فرصة استثمارية و((عرض لا يعوض)) امتلك استديو ت...,ابو الخير,( 18 ),https://www.google.com/maps/search/?api=1&quer...,32.024002,...,1,Furnished,30 meter square,Ground Floor,6 - 9 years,Double Glazed Windows,NaN,NaN,NaN,NaN
1498,https://opensooq.com/en/search/241989049/mixed...,241989049.0,ارض في ضاحية الصفوه النموذجيه,['https://opensooq-images.os-cdn.com/previews/...,21-11-2010,يقع المشروع عند أقرب نقطه من مدينة عمان الجديد...,الثقه العقاريه,( 10 ),https://www.google.com/maps/search/?api=1&quer...,31.795315,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
